### Object Detection using webcam 

###### Highly Recommanded that using "Jupyter Notebook" because you camera acesses only by jupyter

In [15]:
import cv2
import numpy as np

# Load YOLO model
net = cv2.dnn.readNetFromDarknet("yolov3.cfg", "yolov3.weights")

# Get the names of all layers in the network
layer_names = net.getLayerNames()

# Get the output layer names
unconnected_out_layers = net.getUnconnectedOutLayers()

# Check if the output is a 2D array or a 1D array/scalar
if len(unconnected_out_layers.shape) == 2:
    output_layers = [layer_names[i[0] - 1] for i in unconnected_out_layers]
else:
    output_layers = [layer_names[i - 1] for i in unconnected_out_layers]

# Generate random colors for each class
colors = np.random.uniform(0, 255, size=(len(classes), 3))

print("Output layers:", output_layers)

Output layers: ['yolo_82', 'yolo_94', 'yolo_106']


In [21]:
video = cv2.VideoCapture(0)


while True:

    ret,img=video.read()
    height, width, channels = img.shape
    # Detecting objects

    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)

    outs = net.forward(output_layers)
    # Showing informations on the screen

    class_ids = []

    confidences = []

    boxes = []

    for out in outs:

        for detection in out:

            scores = detection[5:]

            class_id = np.argmax(scores)

            confidence = scores[class_id]

            if confidence > 0.5:

                # Object detected

                center_x = int(detection[0] * width)

                center_y = int(detection[1] * height)

                w = int(detection[2] * width)

                h = int(detection[3] * height)

                # Rectangle coordinates

                x = int(center_x - w / 2)

                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])

                confidences.append(float(confidence))

                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    #print(indexes)

    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):

        if i in indexes:

            x, y, w, h = boxes[i]

            label = str(classes[class_ids[i]])

            color = colors[i]

            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)

            cv2.putText(img, label, (x, y + 30), font, 3, color, 2)    

    #frame = cv2.resize(frame, None, fx=2, fy=2)
    cv2.imshow("running video", img)
    
    if cv2.waitKey(30) & 0xFF==ord('q'):
        break
video.release()
cv2.destroyAllWindows()